# Mission 1: 오염원 첫 발견 - 굴뚝 위치 탐지 (Object Detection)

- 작성자: 인하오남매
- 제출 파일: Mission1.ipynb
- 환경: Google Colab (Python3 + A100 GPU)


본 파일은 JSON→YOLO 라벨 변환, 전처리/분할, 학습, 추론을 수행

## [1] 환경 준비
- Google Drive 마운트
- `requirements.txt` 설치
- `ultralytics` 설치
- 라이브러리 import
- GPU/Device 확인
- Seed 설정

In [1]:
# === Google Drive 마운트 ===
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/M1

/content/drive/MyDrive/M1


In [3]:
# === requirements.txt 설치 ===
!pip install -r requirements.txt -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.4 MB/s eta 0:00:00


In [4]:
# === ultralytics 설치 ===
!pip install ultralytics -q

In [5]:
# === 라이브러리 import ===

# 표준 라이브러리
import json
import os
import random
import shutil

# 서드파티
import torch
from ultralytics import YOLO

# 로컬/프로젝트
from pathlib import Path

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [6]:
# === GPU/Device 설정 ===
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print('Using PyTorch:', torch.__version__, '| Device:', DEVICE)

Using PyTorch: 2.8.0+cu126 | Device: cuda


In [7]:
# === Seed 고정 ===
SEED = 42

random.seed(SEED)                          # Python random 모듈 시드
torch.manual_seed(SEED)                    # PyTorch CPU 및 현재 GPU 연산 시드
torch.backends.cudnn.deterministic = True  # CuDNN 연산 결정론적 수행
torch.backends.cudnn.benchmark = False     # 입력 크기가 달라도 결과 동일

In [8]:
# === 공용 유틸 ===
def ensure_dir(p: Path) -> Path:
    """
    지정한 경로에 폴더가 없으면 생성하고, Path 객체를 반환하는 함수

    Args:
        p (Path): 생성할 폴더 경로

    Returns:
        p (Path): 생성된(또는 이미 존재하는) 폴더 경로
    """
    p.mkdir(parents=True, exist_ok=True)
    return p

## [2] 데이터 경로 설정
- **TS_KS**: 학습용 원본 이미지 전체
- **VL_KS**: 최종 추론 대상
- **TL_KS_BBOX**: TS_KS 라벨(JSON)


In [9]:
# === 데이터 경로 설정 ===

DATASET_ROOT = Path("./data")

# 원본 이미지
TSKS_IMAGES_PATH = DATASET_ROOT / "images" / "TS_KS"
IMAGES_TEST      = DATASET_ROOT / "images" / "test"

# 어노테이션
TLKS_ANNOS_PATH = DATASET_ROOT / "annotations" / "TL_KS_BBOX"
TEST_ANNOS_PATH = DATASET_ROOT / "annotations" / "test"

# YOLO 라벨
LABELS_OUT_TRAIN = DATASET_ROOT / "labels" / "TL_KS_BBOX"
LABELS_OUT_TEST  = DATASET_ROOT / "labels" / "test"

# 전처리 이후 학습용 표준 구조 (YOLO 포맷)
IMAGES_TRAIN = DATASET_ROOT / "images" / "train"
IMAGES_VAL   = DATASET_ROOT / "images" / "val"
LABELS_TRAIN = DATASET_ROOT / "labels" / "train"
LABELS_VAL   = DATASET_ROOT / "labels" / "val"

## [3] 학습 하이퍼파라미터 설정
- 데이터 분할 비율
- 학습 및 추론에 공통 적용되는 파라미터

In [10]:
# === 하이퍼파라미터 설정 ===

# 데이터 분할 설정
VAL_RATIO = 0.10  # 9:1

# 학습 설정
PROJECT_ROOT = Path("./runs")   # 결과 저장 경로
EXP_NAME     = "yolov8m"        # 실험(학습) 이름
INFER_NAME   = "infer_yolov8m"  # 추론(Inference) 결과 저장 이름
EVAL_NAME    = "eval_yolov8m"   # 성능 평가(Evaluation) 결과 저장 이름

MODEL    = "yolov8m.pt"
IMGSZ    = 800                 # 입력 이미지 크기
EPOCHS   = 40                  # 학습 epoch 수
BATCH    = 16                  # 배치 사이즈
PATIENCE = 15                  # 조기 종료 기준

# 추론 설정 (대회 규정)
CONF_TH = 0.25                 # confidence threshold
IOU_TH  = 0.50                 # IoU threshold
MAX_DET = 100                  # 이미지당 최대 검출 객체 수

# 전처리 후 생성되는 dataset.yaml 경로
DATA_YAML = DATASET_ROOT / "dataset.yaml"

## [4] 전처리
- 원본 어노테이션(JSON, `annotations/…`)을 **YOLO txt**(`labels/…`)로 변환
- **TS_KS**(학습 원본)를 **train/val = 9:1**로 분할(복사)
- `dataset.yaml` 자동 생성


In [11]:
# === 1. JSON → YOLO txt ===
def convert_json_to_yolo(json_dir: Path, labels_out: Path) -> dict:
    """
    VIA JSON 어노테이션을 YOLO 형식(txt)으로 변환하는 함수.

    각 rect(x, y, width, height) 정보를 YOLO 포맷(class x_center y_center width height)으로 변환하여 저장한다.
    모든 객체의 클래스는 0으로 고정된다.

    Args:
        json_dir (Path): VIA json 파일들이 위치한 폴더 경로
        labels_out (Path): 변환된 YOLO txt 라벨 파일 저장 경로

    Returns:
        dict: 변환 결과 요약 정보
            - json_files (int): 처리한 JSON 파일 개수
            - instances (int): 총 객체(굴뚝) 수
            - txt_written (int): 변환된 txt 파일 개수
    """
    stats = dict(json_files=0, instances=0, txt_written=0)
    ensure_dir(labels_out)

    # 변환 대상 JSON 목록
    json_files = sorted(json_dir.glob("*.json"))
    for jf in json_files:
        stats["json_files"] += 1
        try:
            data = json.loads(jf.read_text(encoding="utf-8"))
        except Exception as e:
            print(f"JSON 파싱 실패: {jf.name} ({e})")
            continue

        # JSON 내의 이미지 항목 루프
        for item in data.values():
            fn = item.get("filename")
            fa = item.get("file_attributes", {})
            iw, ih = int(fa.get("img_width")), int(fa.get("img_height"))  # 이미지 크기

            regions = item.get("regions", [])
            stats["instances"] += len(regions)

            lines = []
            for reg in regions:
                # rect 기반 bbox → YOLO 형식으로 변환
                sa = reg.get("shape_attributes", {})
                x = float(sa["x"]); y = float(sa["y"])           # 좌상단 좌표
                w = float(sa["width"]); h = float(sa["height"])  # bbox 크기

                # 중심 좌표 계산
                cx = x + w/2.0
                cy = y + h/2.0

                # 정규화 (0~1 범위)
                nx = cx / iw
                ny = cy / ih
                nw = w / iw
                nh = h / ih

                # YOLO 형식: class_id cx cy w h
                lines.append(f"0 {nx:.6f} {ny:.6f} {nw:.6f} {nh:.6f}")

            # YOLO txt 저장
            out_path = labels_out / f"{Path(fn).stem}.txt"
            out_path.write_text("\n".join(lines), encoding="utf-8")
            stats["txt_written"] += 1

    # 변환 결과 요약
    print(f"[Convert] {json_dir.name}: json={stats['json_files']}, txt={stats['txt_written']}, instances={stats['instances']}")
    return stats

In [12]:
# === 2. Split ===
def split_tsks_to_train_val(val_ratio: float) -> None:
    """
    TS_KS 이미지를 train/val로 분할하는 함수
    동일 stem의 txt 라벨도 함께 복사합니다.

    원본 보존을 위해 '복사'로 진행합니다.

    Args:
        val_ratio (float): 검증 세트 비율 (기본 0.1 → 9:1 분할)
    """
    print("[Split by Copy] TS_KS → Train/Val")

    # 대상 폴더 보장
    ensure_dir(IMAGES_TRAIN); ensure_dir(IMAGES_VAL)
    ensure_dir(LABELS_TRAIN); ensure_dir(LABELS_VAL)

    # 이미지 목록 셔플
    tsks_images = sorted(TSKS_IMAGES_PATH.glob("*.jpg"))
    random.shuffle(tsks_images)

    n_total     = len(tsks_images)
    n_val       = int(round(n_total * val_ratio))
    train_set   = set(tsks_images[n_val:])
    val_set     = set(tsks_images[:n_val])

    # 복사
    def _safe_copy(img_path: Path, dst_img_dir: Path, dst_lbl_dir: Path):
        """ 이미지와 동일 stem의 라벨(txt)을 함께 복사하는 함수"""
        lbl_src = LABELS_OUT_TRAIN / f"{img_path.stem}.txt"

        img_dst = dst_img_dir / img_path.name
        lbl_dst = dst_lbl_dir / lbl_src.name

        shutil.copy2(str(img_path), str(img_dst))  # 이미지 복사
        shutil.copy2(str(lbl_src), str(lbl_dst))   # 라벨 복사

    # 분할
    for src in train_set:
        _safe_copy(src, IMAGES_TRAIN, LABELS_TRAIN)
    for src in val_set:
        _safe_copy(src, IMAGES_VAL, LABELS_VAL)

    # 복사된 개수
    n_train_img = len(list(IMAGES_TRAIN.glob("*.jpg")))
    n_val_img   = len(list(IMAGES_VAL.glob("*.jpg")))
    n_train_lbl = len(list(LABELS_TRAIN.glob("*.txt")))
    n_val_lbl   = len(list(LABELS_VAL.glob("*.txt")))

    print(f"[Split by Copy] 완료: 전체 이미지 수={n_total}")
    print(f" expected split → train={len(train_set)}, val={len(val_set)} (val_ratio={val_ratio:.2f})")
    print(f" copied result  → train: images={n_train_img}, labels={n_train_lbl} | val: images={n_val_img}, labels={n_val_lbl}")

In [13]:
# === 3. dataset.yaml 생성 ===
def write_dataset_yaml(data_yaml: Path, names: list[str]) -> None:
    """
    YOLO 학습용 dataset.yaml 파일을 생성하는 함수

    Args:
        data_yaml (Path): 생성할 dataset.yaml 파일 경로
        names (list[str]): 클래스 이름 리스트
    """
    # yaml 작성
    yaml_text = (
        f"train: {IMAGES_TRAIN.resolve()}\n"
        f"val: {IMAGES_VAL.resolve()}\n"
        f"test: {IMAGES_TEST.resolve()}\n"
        f"nc: {len(names)}\n"
        f"names: {names}\n"
    )

    data_yaml.write_text(yaml_text, encoding="utf-8")
    print(f"[YAML] 저장 완료: {data_yaml.resolve()}")

## [5] 모델 학습
- 사전 학습된 모델(`yolov8m.pt`)을 불러와 fine-tuning 진행
- 입력 크기(`imgsz=800`), 학습 `epoch=40`, `batch=16`
- 학습 결과는 지정한 `project_root/exp_name` 경로에 저장됩니다.

In [14]:
# === 학습 함수 정의 ===
def train_yolov8() -> None:
    """
    전역 하이퍼파라미터(MODEL, DATA_YAML, EPOCHS, IMGSZ, BATCH, SEED, PROJECT_ROOT, EXP_NAME)를 사용하여 YOLOv8 모델을 학습하는 함수
    결과는 PROJECT_ROOT/EXP_NAME 폴더에 저장됩니다.
    """
    # 모델 로드 (사전 학습된 가중치)
    model = YOLO(MODEL)

    # 학습 실행
    model.train(
        data=str(DATA_YAML),
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        patience=PATIENCE,
        seed=SEED,
        device=DEVICE,
        project=str(PROJECT_ROOT),
        name=EXP_NAME
    )

    print(f"[Train] {EXP_NAME} 완료 → 결과 저장: {PROJECT_ROOT / EXP_NAME}")

## [6] 추론(Inference)
- 입력: Test 이미지 경로
- 출력: `PROJECT_ROOT/INFER_NAME/` 경로에 시각화 이미지, 라벨(txt), JSON 결과가 저장
- 추론 설정은 대회 규정에 따라 고정 (`CONF_TH=0.25`, `IOU_TH=0.50`, `MAX_DET=100`)

In [15]:
# === 가중치/모델 로드 (추론용) ===
def get_best_weights() -> Path:
    """
    학습된 best.pt 경로를 반환하는 함수

    Returns:
        weights (path): 학습된 best.pt 경로
    """
    weights = PROJECT_ROOT / EXP_NAME / "weights" / "best.pt"
    return weights

def load_trained_model() -> YOLO:
    """
    학습된 best.pt 모델을 로드하는 함수

    Returns:
        model (YOLO): 학습된 모델
    """
    weights = get_best_weights()
    model = YOLO(str(weights))
    return model

In [16]:
# === 추론 함수 정의 ===
def infer_yolov8() -> None:
  """
  전역 하이퍼파라미터를 사용하여 YOLOv8 모델을 추론하는 함수
  결과는 PROJECT_ROOT/INFER_NAME 폴더에 저장됩니다.
  """
  # 모델 로드 (학습된 best.pt)
  model = load_trained_model()

  # 추론 실행
  model.predict(
      source=str(IMAGES_TEST),
      imgsz=IMGSZ,
      conf=CONF_TH,
      iou=IOU_TH,
      max_det=MAX_DET,
      device=DEVICE,
      save=True, save_txt=True, save_conf=True,
      project=str(PROJECT_ROOT),
      name=INFER_NAME,
      verbose=False
  )

  print(f"[Infer] {INFER_NAME} 완료 → 결과 저장: {PROJECT_ROOT / INFER_NAME}")

## [7] 성능 평가
- 검증 데이터셋(`test(VS_KS)` + `test(VL_KS_BBOX)`)을 이용해 학습된 모델을 평가
- 결과는 `PROJECT_ROOT/EVAL_NAME/`에 저장

In [17]:
# === 성능 평가 함수 정의 ===
def evaluate_yolov8() -> None:
    """
    dataset.yaml의 test(VS_KS) 경로를 사용하여 YOLOv8 모델을 평가하는 함수
    결과는 PROJECT_ROOT/EVAL_NAME 폴더에 저장됩니다.
    """
    # 모델 로드 (학습된 best.pt)
    model = load_trained_model()

    # 추론 실행
    model.val(
        data=str(DATA_YAML),
        split='test',
        imgsz=IMGSZ,
        conf=CONF_TH,
        iou=IOU_TH,
        max_det=MAX_DET,
        device=DEVICE,
        project=str(PROJECT_ROOT),
        name=EVAL_NAME,
        verbose=True,
        plots=True
    )

    print(f"[Eval] {EVAL_NAME} 완료 → 결과 저장: {PROJECT_ROOT / EVAL_NAME}")


## [8] 메인 실행
1. 전처리 (JSON → YOLO 변환, Split, dataset.yaml 생성)
2. 모델 학습
3. 추론
4. 성능 평가

In [18]:
# === Step 1. 전처리 ===

# JSON → YOLO 변환
convert_json_to_yolo(TLKS_ANNOS_PATH, LABELS_OUT_TRAIN)
convert_json_to_yolo(TEST_ANNOS_PATH, LABELS_OUT_TEST)

# Split
split_tsks_to_train_val(val_ratio=VAL_RATIO)

# dataset.yaml 생성
write_dataset_yaml(DATA_YAML, names=["chimney"])

[Convert] TL_KS_BBOX: json=8052, txt=8052, instances=10619
[Convert] test: json=1006, txt=1006, instances=1322
[Split by Copy] TS_KS → Train/Val
[Split by Copy] 완료: 전체 이미지 수=8052
 expected split → train=7247, val=805 (val_ratio=0.10)
 copied result  → train: images=7247, labels=7247 | val: images=805, labels=805
[YAML] 저장 완료: /content/drive/MyDrive/M1/data/dataset.yaml


In [19]:
# === Step 2. 학습 ===
ensure_dir(PROJECT_ROOT) # 실행 전 결과 저장 폴더 보장
train_yolov8()

New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m, nbs=64, nms=False, opset=None, optimize=Fal

In [20]:
# === Step 3. 추론 ===
infer_yolov8()

WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs/infer_yolov8m
1003 labels saved to runs/infer_yolov8m/labels
[Infer] infer_yolov8m 완료 → 결과 저장: runs/infer_yolov8m


In [21]:
# === Step 4. 성능 평가 ===
evaluate_yolov8()

Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access ✅ (ping: 0.3±0.1 ms, read: 62.9±24.1 MB/s, size: 112.7 KB)
val: Scanning /content/drive/MyDrive/M1/data/labels/test... 1006 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1006/1006 270.6it/s 3.7s
val: New cache created: /content/drive/MyDrive/M1/data/labels/test.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 63/63 7.6it/s 8.3s
                   all       1006       1322      0.969      0.983      0.988      0.869
Speed: 0.5ms preprocess, 3.7ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/eval_yolov8m
[Eval] eval_yolov8m 완료 → 결과 저장: runs/eval_yolov8m
